In [0]:
file_location = r"/mnt/staging/people MAP.txt"
file_type = "csv"


infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


currentschema = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
columns  = [col.strip() for col in currentschema.columns]
currentschema = currentschema.toDF(*columns)

display(currentschema)

currentschema.printSchema()

ID,FirstName,MiddleName,LastName,Gender,Birthyear,salary,Pet,food
123,jack,qwe,dfg,m,99,2000,cat,veg
123,jack,qwe,dfg,m,99,2000,tiger,veg
123,jack,qwe,dfg,m,99,2000,dog,veg
124,ram,asd,cvb,f,96,1000,cat,non-veg
125,job,zxc,ert,m,55,3000,mouse,non-veg
126,steve,rty,tyu,m,85,4000,dog,veg
126,steve,rty,tyu,m,85,4000,lion,veg
126,steve,rty,tyu,m,85,4000,camel,veg
127,mike,fgh,hjk,f,75,5000,tiger,veg
128,sham,vbn,fds,f,69,6000,elephant,veg


root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- Birthyear: integer (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- food: string (nullable = true)

In [0]:
file_location = r"/mnt/staging/fnamemissing overflow1.txt"
file_type = "csv"


infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


fnamemissing_overflow = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
columns  = [col.strip() for col in fnamemissing_overflow.columns]
fnamemissing_overflow = fnamemissing_overflow.toDF(*columns)

display(fnamemissing_overflow)

ID,F_Name,M_Name,L_Name,Gender,salary,Pet,state,colour
123,jack,qwe,dfg,m,2000,cat,mh,B
123,jack,qwe,dfg,m,2000,tiger,up,A
123,jack,qwe,dfg,m,2000,dog,tn,A
124,ram,asd,cvb,f,1000,cat,tn,A
125,job,zxc,ert,m,3000,mouse,tn,A
126,steve,rty,tyu,m,4000,dog,up,A
126,steve,rty,tyu,m,4000,lion,jk,A
126,steve,rty,tyu,m,4000,camel,jk,S
127,mike,fgh,hjk,f,5000,tiger,jk,D
128,sham,vbn,fds,f,6000,elephant,jk,D


In [0]:
file_location = r"/mnt/staging/CRF-Details_CSV.csv"
file_type = "csv"

infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

mappingfile = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.option("header", first_row_is_header) \
.option("sep", delimiter) \
.load(file_location)

display(mappingfile)
mapping = {row['ColumnNamesReceived']:row['ColumnNames'] for row in mappingfile.collect()}


print(mapping)

ColumnNames,ColumnNamesReceived
FirstName,FName
MiddleName,MName
LastName,LNAME
FirstName,F_Name
MiddleName,M_Name
LastName,L_Name


{'FName': 'FirstName', 'MName': 'MiddleName', 'LNAME': 'LastName', 'F_Name': 'FirstName', 'M_Name': 'MiddleName', 'L_Name': 'LastName'}

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import MapType, StringType
from itertools import chain
def parsingdataframe(df,currentschema,mapping):
    
    #Creating dataframe 
    s1 = spark.createDataFrame(currentschema.dtypes, ["d1_name", "d1_type"])
    print("s1")
    print(s1)
    s2 = spark.createDataFrame(df.dtypes, ["d2_name", "d2_type"])
    print("s2")
    print(s2)
    mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])
    print("this")
    print(mapping_expr)
    s2 = s2.withColumn("value", mapping_expr.getItem(col("d2_name")))

    s2 = s2.withColumn("d2_name",when(s2.value.isNotNull(), s2.value).otherwise(s2.d2_name))
    s2.show()
    row_list = s2.select('d2_name').collect()
    mappingColumns = [row.d2_name for row in row_list]
    df = df.toDF(*mappingColumns)
   
    s2 = spark.createDataFrame(df.dtypes, ["d2_name", "d2_type"])
    
  
    difference = (
            s1.join(s2, s1.d1_name == s2.d2_name, how="outer")
            
        ).select("d2_name","d1_name")

    #for overflow columns
    diff = difference.filter(difference.d1_name.isNull())
    row_list = diff.select('d2_name').collect()
    overflowColumns = [row.d2_name for row in row_list]
    
    #for missing column
    diff = difference.filter(difference.d2_name.isNull())
    
    row_list = diff.select('d1_name').collect()
    missingColumns = [row.d1_name for row in row_list]
    metric = create_map(list(chain(*(
   (lit(name), col(name)) for name in overflowColumns))))
    
    if overflowColumns:
        
           
        df=df.withColumn("ClientSpecificField",metric)
        
        for new_col in overflowColumns:
            
            
            df =df.drop(new_col)
    if missingColumns:
        for new_col in missingColumns:
            df = df.withColumn(new_col, lit(None).cast('string'))
       
    return df
        
    

In [0]:
df_new = parsingdataframe(fnamemissing_overflow,currentschema,mapping)

df_new.display()



s1
DataFrame[d1_name: string, d1_type: string]
s2
DataFrame[d2_name: string, d2_type: string]
this
Column<'map(FName, FirstName, MName, MiddleName, LNAME, LastName, F_Name, FirstName, M_Name, MiddleName, L_Name, LastName)'>
+----------+-------+----------+
 d2_name|d2_type| value|
+----------+-------+----------+
 ID| int| null|
 FirstName| string| FirstName|
MiddleName| string|MiddleName|
 LastName| string| LastName|
 Gender| string| null|
 salary| int| null|
 Pet| string| null|
 state| string| null|
 colour| string| null|
+----------+-------+----------+

ID,FirstName,MiddleName,LastName,Gender,salary,Pet,ClientSpecificField,Birthyear,food
123,jack,qwe,dfg,m,2000,cat,"Map(colour -> B , state -> mh)",null,null
123,jack,qwe,dfg,m,2000,tiger,"Map(colour -> A , state -> up)",null,null
123,jack,qwe,dfg,m,2000,dog,"Map(colour -> A , state -> tn)",null,null
124,ram,asd,cvb,f,1000,cat,"Map(colour -> A , state -> tn)",null,null
125,job,zxc,ert,m,3000,mouse,"Map(colour -> A , state -> tn)",null,null
126,steve,rty,tyu,m,4000,dog,"Map(colour -> A , state -> up)",null,null
126,steve,rty,tyu,m,4000,lion,"Map(colour -> A , state -> jk)",null,null
126,steve,rty,tyu,m,4000,camel,"Map(colour -> S , state -> jk)",null,null
127,mike,fgh,hjk,f,5000,tiger,"Map(colour -> D , state -> jk)",null,null
128,sham,vbn,fds,f,6000,elephant,"Map(colour -> D, state -> jk)",null,null


In [0]:
df_new.display()
df_new.printSchema()

ID,FirstName,MiddleName,LastName,Gender,salary,Pet,ClientSpecificField,Birthyear,food
123,jack,qwe,dfg,m,2000,cat,"Map(colour -> B , state -> mh)",null,null
123,jack,qwe,dfg,m,2000,tiger,"Map(colour -> A , state -> up)",null,null
123,jack,qwe,dfg,m,2000,dog,"Map(colour -> A , state -> tn)",null,null
124,ram,asd,cvb,f,1000,cat,"Map(colour -> A , state -> tn)",null,null
125,job,zxc,ert,m,3000,mouse,"Map(colour -> A , state -> tn)",null,null
126,steve,rty,tyu,m,4000,dog,"Map(colour -> A , state -> up)",null,null
126,steve,rty,tyu,m,4000,lion,"Map(colour -> A , state -> jk)",null,null
126,steve,rty,tyu,m,4000,camel,"Map(colour -> S , state -> jk)",null,null
127,mike,fgh,hjk,f,5000,tiger,"Map(colour -> D , state -> jk)",null,null
128,sham,vbn,fds,f,6000,elephant,"Map(colour -> D, state -> jk)",null,null


root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- ClientSpecificField: map (nullable = false)
 |-- key: string
 |-- value: string (valueContainsNull = true)
-- Birthyear: string (nullable = true)
-- food: string (nullable = true)

In [0]:
#df_new.select('ClientSpecificField.*')
#display(df_new.select('*', 'ClientSpecificField.*'))


File "<command-2463731506927115>" , line 7 
 f.struct(*[f.col("ClientSpecificField").getItem(c)])) 
 ^
 SyntaxError : unexpected EOF while parsing

In [0]:
df_new.select('ClientSpecificField').dtypes

Out[43]: [('ClientSpecificField', 'map<string,string>')]

In [0]:
import pandas as pd
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:10] == 'map<string,string>'] 
    #Square bracket means creating a list, c[0] first item has the position 0

In [0]:
#flatten_df(df_new)

#flat_cols = [c[0] for c in df_new.dtypes if c[1][:6] == 'string'] # Working
flat_cols = [c[0] for c in df_new.dtypes if c[1][:10] == 'map'] 
print(flat_cols)

#df_2 = df_new.dtypes

#display(df_2)

[]

In [0]:
course_df2 = df_new.withColumn("ClientSpecificField", 
                                  df_new["ClientSpecificField"]
                                  .cast('string'))
course_df2.printSchema()
display(course_df2)

root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- ClientSpecificField: string (nullable = false)
-- Birthyear: string (nullable = true)
-- food: string (nullable = true)

ID,FirstName,MiddleName,LastName,Gender,salary,Pet,ClientSpecificField,Birthyear,food
123,jack,qwe,dfg,m,2000,cat,"{colour -> B , state -> mh}",null,null
123,jack,qwe,dfg,m,2000,tiger,"{colour -> A , state -> up}",null,null
123,jack,qwe,dfg,m,2000,dog,"{colour -> A , state -> tn}",null,null
124,ram,asd,cvb,f,1000,cat,"{colour -> A , state -> tn}",null,null
125,job,zxc,ert,m,3000,mouse,"{colour -> A , state -> tn}",null,null
126,steve,rty,tyu,m,4000,dog,"{colour -> A , state -> up}",null,null
126,steve,rty,tyu,m,4000,lion,"{colour -> A , state -> jk}",null,null
126,steve,rty,tyu,m,4000,camel,"{colour -> S , state -> jk}",null,null
127,mike,fgh,hjk,f,5000,tiger,"{colour -> D , state -> jk}",null,null
128,sham,vbn,fds,f,6000,elephant,"{colour -> D, state -> jk}",null,null


In [0]:
#flatten_df(df_new)

#flat_cols = [c[0] for c in df_new.dtypes if c[1][:6] == 'string'] # Working
flat_cols = [c[0] for c in course_df2.dtypes if c[1][:10] == 'string'] 
print(flat_cols)

#df_2 = df_new.dtypes

#display(df_2)

['FirstName', 'MiddleName', 'LastName', 'Gender', 'Pet', 'ClientSpecificField', 'Birthyear', 'food']

In [0]:
import pandas as pd
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:10] == 'string']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:10] == 'string']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [0]:

import pandas as pd
import pyspark.sql.functions as F

flat_cols = [c[0] for c in course_df2.dtypes if c[1][:10] == 'string']
nested_cols = [c[0] for c in course_df2.dtypes if c[1][:10] == 'string']

flat_df = course_df2.select('flat_cols*'')

print(flat_df)


File "<command-2463731506927114>" , line 7 
 flat_df = course_df2.select('flat_cols*'') 
 ^
 SyntaxError : EOL while scanning string literal

In [0]:
df_2new = flatten_df()

display(df_2new)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2463731506927113> in <module> 
 ----> 1 df_2new = flatten_df ( course_df2 ) 
 2 
 3 display ( df_2new ) 

 <command-2463731506927112> in flatten_df (nested_df) 
 7 
 8 flat_df = nested_df.select(flat_cols +
 ----> 9 [F.col(nc+'.'+c).alias(nc+'_'+c)
 10 for nc in nested_cols
 11 for c in nested_df.select(nc+'.*').columns])

 <command-2463731506927112> in <listcomp> (.0) 
 9 [F.col(nc+'.'+c).alias(nc+'_'+c)
 10 for nc in nested_cols
 ---> 11 for c in nested_df.select(nc+'.*').columns])
 12 return flat_df

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Can only star expand struct data types. Attribute: `ArrayBuffer(FirstName)`

In [0]:
from pyspark.sql.functions import explode
##df_new.display()

keys = (df_new
    .select(explode("ClientSpecificField"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

print(keys)

from pyspark.sql.functions import col

exprs = [col("ClientSpecificField").getItem(k).alias(k) for k in keys]

print("Next Dataframe")

df_new2 = df_new.select(*exprs)
df_new2.show()
#df2_new = df_new.select(df_new.ClientSpecificField,explode(df_new.ClientSpecificField))
#df2_new.printSchema()
#df2_new.show()

#df2_new = spark.createDataFrame(data=df_new, schema = ['name','knownLanguages','properties'])
#df2_new.printSchema()
#df2_new.show()

['colour', 'state']
Next Dataframe
+------+-----+
colour|state|
+------+-----+
 B | mh|
 A | up|
A | tn|
 A | tn|
 A | tn|
 A | up|
 A | jk|
 S | jk|
 D | jk|
 D| jk|
 F| up|
+------+-----+

In [0]:
#merged_df = df_new.unionByName(df_new2, allowMissingColumns=True)
#merged_df.display()

result = df_new.crossJoin(df_new2)
result1 = result.drop("ClientSpecificField")
result1.display()
result1.printSchema()

ID,FirstName,MiddleName,LastName,Gender,salary,Pet,Birthyear,food,colour,state
123,jack,qwe,dfg,m,2000,cat,null,null,B,mh
123,jack,qwe,dfg,m,2000,cat,null,null,A,up
123,jack,qwe,dfg,m,2000,cat,null,null,A,tn
123,jack,qwe,dfg,m,2000,cat,null,null,A,tn
123,jack,qwe,dfg,m,2000,cat,null,null,A,tn
123,jack,qwe,dfg,m,2000,cat,null,null,A,up
123,jack,qwe,dfg,m,2000,cat,null,null,A,jk
123,jack,qwe,dfg,m,2000,cat,null,null,S,jk
123,jack,qwe,dfg,m,2000,cat,null,null,D,jk
123,jack,qwe,dfg,m,2000,cat,null,null,D,jk


root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- Birthyear: string (nullable = true)
-- food: string (nullable = true)
-- colour: string (nullable = true)
-- state: string (nullable = true)

In [0]:
result1.printSchema()

root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- Birthyear: string (nullable = true)
-- food: string (nullable = true)
-- colour: string (nullable = true)
-- state: string (nullable = true)

In [0]:
#result1. write. options (header ='True', delimiter =',') . csv ("/mnt/output/datacsv1")


In [0]:
df_new.display()
df_new.printSchema()

ID,FirstName,MiddleName,LastName,Gender,salary,Pet,ClientSpecificField,Birthyear,food
123,jack,qwe,dfg,m,2000,cat,"Map(colour -> B , state -> mh)",null,null
123,jack,qwe,dfg,m,2000,tiger,"Map(colour -> A , state -> up)",null,null
123,jack,qwe,dfg,m,2000,dog,"Map(colour -> A , state -> tn)",null,null
124,ram,asd,cvb,f,1000,cat,"Map(colour -> A , state -> tn)",null,null
125,job,zxc,ert,m,3000,mouse,"Map(colour -> A , state -> tn)",null,null
126,steve,rty,tyu,m,4000,dog,"Map(colour -> A , state -> up)",null,null
126,steve,rty,tyu,m,4000,lion,"Map(colour -> A , state -> jk)",null,null
126,steve,rty,tyu,m,4000,camel,"Map(colour -> S , state -> jk)",null,null
127,mike,fgh,hjk,f,5000,tiger,"Map(colour -> D , state -> jk)",null,null
128,sham,vbn,fds,f,6000,elephant,"Map(colour -> D, state -> jk)",null,null


root
-- ID: integer (nullable = true)
-- FirstName: string (nullable = true)
-- MiddleName: string (nullable = true)
-- LastName: string (nullable = true)
-- Gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Pet: string (nullable = true)
-- ClientSpecificField: map (nullable = false)
 |-- key: string
 |-- value: string (valueContainsNull = true)
-- Birthyear: string (nullable = true)
-- food: string (nullable = true)

In [0]:
import pandas as pd
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct'] 
    #Square bracket means creating a list, c[0] first item has the position 0
    
    
    

In [0]:
df_2 = flatten_df()

display(df_2)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2463731506927105> in <module> 
 ----> 1 df_2 = flatten_df ( ) 
 2 
 3 display ( df_2 ) 

 TypeError : flatten_df() missing 1 required positional argument: 'nested_df'

In [0]:
#Code to flatten the Map struct column
#mydf = df_new.select(col("ID","ClientSpecificField"))
#mydf = df_new["ID", "FirstName", "MiddleName", "LastName", "Gender", "salary", "Pet", "ClientSpecificField.*", "Birthyear", "food"] # Not working
mydf = df_new["ID", "FirstName", "MiddleName", "LastName", "Gender", "salary", "Pet", "ClientSpecificField.colour", "ClientSpecificField.state", "Birthyear", "food"]
display(mydf)

#import spark.implicits._
#val df = mydf.toDF(columns:_*)
#df.show(false)

In [0]:
pip uninstall pyspark

In [0]:
%scala
import scala.collection.mutable
import scala.util.control.NonFatal

import org.apache.spark.sql._
import org.apache.spark.sql.catalyst.analysis.{Resolver, UnresolvedAttribute}
import org.apache.spark.sql.catalyst.util.CaseInsensitiveMap
import org.apache.spark.sql.functions.{col, struct}
import org.apache.spark.sql.types._
val data = Seq(
  Row(Row("this", "is"), "something", "cool", ";)")
)

val schema = StructType(
  Seq(
    StructField(
      "foo",
      StructType(
        Seq(
          StructField("bar", StringType, true),
          StructField("baz", StringType, true)
        )
      ),
      true
    ),
    StructField("x", StringType, true),
    StructField("y", StringType, true),
    StructField("z", StringType, true)
  )
)

val df = spark.createDataFrame(
  spark.sparkContext.parallelize(data),
  StructType(schema)
)

display(df)
df.printSchema

In [0]:
import pandas as pd
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [0]:
df_2 = flatten_df(df_new)

display(df_2)
